In [1]:
import scipy
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import os, sys
rootpath = os.path.join(os.getcwd(), '.')
sys.path.append(rootpath)
from src.simulation import *
from src.approximation import *
from src.theory import *
# reimport modules if they change
%load_ext autoreload
%autoreload 2

# Numerically evaluate the solution to the Fokker Planck solution

This is done for integer values of the activity $x$ taking the continuum limit in time $t\to 0$, i.e, looking at the instantaneous number of active neurons in the coupled system

In [2]:
# IMPORTANT: this is the resolution and range of lambda for the plots
lams_0 = 1 - 10 ** np.linspace(0, -4, 64 + 1)

The rest should be left untouched, focusing on one example of $\epsilon$ and $\sigma$ in the paper.

In [3]:
# specify system parameters
params = {
    'N': 10000, # number of nodes
    'K': 100, # number of connections per node
    'mu': 0.2, # fraction of nodes that receive input
    'sigma': 0.01, # std of additive Gaussian noise
    'epsilon': 0.1, # error threshold for overlap
}
params['epsilon'] = 0.1
params['sigma'] = 0.01
dt = 1 #fixed time step

filename = f"dat/results_to0_N{params['N']}_K{params['K']}_mu{params['mu']}_epsilon{params['epsilon']}_sigma{params['sigma']}.txt"

In [4]:
?pmf_from_coupled_fokker_planck

Signature: pmf_from_coupled_fokker_planck(params, h, lam)
Docstring:
Solution to the Mean-field coupled Fokker-Planck equations. 
1.1) compute solution of FP equation of the part that receives input assuming a mean-field coupling to the recurrently coupled rest
ight)}$$
from mean-field assumption
$$ x_rest = rac{(1-\mu)\lambda x_in}{1-(1-\mu)\lambda}$$ 

1.2) compute solution of FP equation for the part that does not receive input assuming a mean-field coupling to the input part
ight)}$$
with 
$$ x_in = \murac{N p_\mathrm{ext} + \lambda (1-p_\mathrm{ext}) x_\mathrm{rest}}{1-\mu\lambda(1-p_\mathrm{ext})} $$
2) convolution of the two solutions to obtain the full pmf
File:      ~/git/projects/sahel_finite-observation-dynamic-range/src/theory.py
Type:      function

In [5]:
##SAHEL: Could we integrate this into functions in the src files and have a run_analytic.py ?

# result = analysis_beta_approximation(params_results, list_lambda, args.database)
support_bound = int(params["N"] * (1 + 4 * params["sigma"]))
fp_support = np.arange(-support_bound, support_bound + 1)
fp_pmf = np.zeros_like(fp_support, dtype=np.float64)
fp_pmf_norm = stats.norm.pdf(fp_support, 0, params["N"] * params["sigma"])


def fp_pmf_noise(lam, h):
    x, pmf = pmf_from_coupled_fokker_planck(params, h=h, lam=lam)
    mask = np.where(np.isin(fp_support, x))
    fp_pmf[mask] = pmf
    # convolution with normal distribution
    return np.convolve(fp_pmf, fp_pmf_norm, mode="same")


def analyse(lam, verbose=False, return_h=False):
    pmf_o_given_h = lambda h: fp_pmf_noise(lam, h)

    # determine h range self-consistently from mean-field solution
    # for low h, assume a population that receives mu*h!
    # a = 1 - (1-lambda*a)(1-p_ext) st. (1-p_ext) = exp(-mu*h) = (1-a)/(1-lambda*a)
    a_min = 0.1*params["sigma"] * params["mu"]
    #h_left = -np.log((1 - a_min) / (1 - lam * params["mu"] * a_min))
    h_left = -np.log((1 - a_min) / (1 - lam * a_min))/params["mu"]
    # for high h, we can assume a_in = a such that a_in = a = 1-(1-lambda*a)(1-p_ext) and (1-p_ext) = exp(-h) = (1-a)/(a-lambda*a)
    a_max = 1 - 0.1*params["sigma"]
    h_right = -np.log((1 - a_max) / (1 - lam * a_max))
    h_range = (h_left, h_right)
    if verbose:
        print(f"lambda: {lam}, h_range: {h_range}")
    if h_range[0] >= h_range[1]:
        return lam, np.nan, np.nan

    # reference distributions
    ref_left = stats.norm.pdf(
        fp_support,
        params["N"] * mean_field_activity(lam, params["mu"], h=0),
        params["N"] * params["sigma"],
    )
    ref_right = stats.norm.pdf(
        fp_support,
        params["N"] * mean_field_activity(lam, params["mu"], h=np.inf),
        params["N"] * params["sigma"],
    )
    pmf_refs = [ref_left, ref_right]

    hs_left = find_discriminable_inputs(
        pmf_o_given_h,
        h_range,
        pmf_refs,
        params["epsilon"],
        start="left",
        verbose=verbose,
    )
    hs_right = find_discriminable_inputs(
        pmf_o_given_h,
        h_range,
        pmf_refs,
        params["epsilon"],
        start="right",
        verbose=verbose,
    )
    if len(hs_left) > 0 and len(hs_right) > 0:
        if return_h:
            return (
                lam,
                0.5 * (len(hs_left) + len(hs_right)),
                dynamic_range((hs_left[0], hs_right[0])),
                hs_left, 
                hs_right,
            )
        else:
            return (
                lam,
                0.5 * (len(hs_left) + len(hs_right)),
                dynamic_range((hs_left[0], hs_right[0])),
            )
    else:
        return lam, np.nan, np.nan

In [6]:
#check for lam=0
analyse(lam=0, verbose=True)

lambda: 0, h_range: (0.0010001000133352234, 6.907755278982136)
possible solution: h=0.1382503364360633 with overlap to end of 2.320625083643081e-18 ... accepted
possible solution: h=0.3000362281999911 with overlap to end of 1.0695878292488262e-13 ... accepted
possible solution: h=0.49444269155849335 with overlap to end of 8.176529956131927e-10 ... accepted
possible solution: h=0.7367477338663413 with overlap to end of 1.1174280362267015e-06 ... accepted
possible solution: h=1.057000616297903 with overlap to end of 0.0002918830676612157 ... accepted
possible solution: h=1.5286608574686036 with overlap to end of 0.015670663967609536 ... accepted
possible solution: h=2.440870070107864 with overlap to end of 0.19217273348529537 ... rejected
possible solution: h=2.0509187269479803 with overlap to end of 2.3168017093885e-18 ... accepted
possible solution: h=1.3520774245701934 with overlap to end of 1.068369993444748e-13 ... accepted
possible solution: h=0.9426841951189597 with overlap to end

(0, 6.0, 11.712822538017637)

In [7]:
# check for lam=0.99
analyse(lam=0.99, verbose=True)

lambda: 0.99, h_range: (1.0001990395668491e-05, 2.3969857684155342)
possible solution: h=0.0017584030902946777 with overlap to end of 5.7616414498947094e-86 ... accepted
possible solution: h=0.004767098645091182 with overlap to end of 3.48854516246586e-77 ... accepted
possible solution: h=0.009211181125672575 with overlap to end of 2.9902023280776675e-68 ... accepted
possible solution: h=0.01533196385975883 with overlap to end of 1.93652147980222e-59 ... accepted
possible solution: h=0.023475923501833987 with overlap to end of 5.818759205605476e-51 ... accepted
possible solution: h=0.034127730156311646 with overlap to end of 5.770031814793997e-43 ... accepted
possible solution: h=0.04796486833427315 with overlap to end of 1.4827332647104854e-35 ... accepted
possible solution: h=0.06594859296212935 with overlap to end of 8.342985260531983e-29 ... accepted
possible solution: h=0.08948211871652771 with overlap to end of 9.15509356067269e-23 ... accepted
possible solution: h=0.120703005707

(0.99, 15.0, 27.378365733056658)

In [8]:
# check for lam=0.9999
analyse(lam=0.9999, verbose=True)

lambda: 0.9999, h_range: (1.0002000272577614e-07, 0.09521926658097167)


possible solution: h=0.0001520911402666212 with overlap to end of 0.0002378227240814731 ... accepted
possible solution: h=0.0017872251580803166 with overlap to end of 0.015222453649935885 ... accepted
possible solution: h=0.017193510385880757 with overlap to end of 0.18918719958502897 ... rejected
possible solution: h=0.008176087326179941 with overlap to end of 8.096300503086297e-27 ... accepted
possible solution: h=0.0008452828142372331 with overlap to end of 7.911336542127317e-05 ... accepted
possible solution: h=8.096479893860252e-05 with overlap to end of 0.23527329369530936 ... rejected


(0.9999, 2.0, 17.30441605451969)

In [9]:
from dask.distributed import Client, LocalCluster, as_completed
# execute independent lambda computations in parallel with dask
cluster = LocalCluster()
dask_client = Client(cluster)

futures = dask_client.map(analyse, lams_0)

# run analysis
data = []
for future in tqdm(as_completed(futures), total=len(lams_0)):
    data.append(future.result())

# sort data by first column
data = np.array(sorted(data, key=lambda x: x[0]))

100%|██████████| 65/65 [50:26<00:00, 46.56s/it]   


In [10]:
data

array([[ 0.        ,  6.        , 11.71282254],
       [ 0.13403568,  7.        , 12.52555317],
       [ 0.25010579,  8.        , 13.31517791],
       [ 0.35061837,  9.        , 14.07835472],
       [ 0.43765867, 10.        , 14.81663696],
       [ 0.51303247, 11.        , 15.53138752],
       [ 0.5783035 , 12.        , 16.22380233],
       [ 0.63482587, 13.        , 16.89498487],
       [ 0.68377223, 14.        , 17.54590445],
       [ 0.72615804, 15.        , 18.17731947],
       [ 0.76286263, 16.        , 18.78993542],
       [ 0.7946475 , 17.        , 19.38424776],
       [ 0.82217206, 17.        , 19.96070874],
       [ 0.84600735, 18.        , 20.51955554],
       [ 0.86664786, 18.        , 21.06099198],
       [ 0.8845218 , 19.        , 21.58492687],
       [ 0.9       , 19.        , 22.0912602 ],
       [ 0.91340357, 20.        , 22.57987901],
       [ 0.92501058, 20.        , 23.05034561],
       [ 0.93506184, 20.        , 23.50226391],
       [ 0.94376587, 20.        , 23.935

In [11]:
os.makedirs(os.path.dirname(filename), exist_ok=True)
# save data to file
np.savetxt(
    filename,
    data,
    delimiter="\t",
    header="#lambda\tnumber of discriminable inputs\tdynamic_range",
    comments="",
)